In [1]:
# !pip install yfpy

In [2]:
import pandas as pd

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")

In [4]:
from yfpy.query import YahooFantasySportsQuery

query = YahooFantasySportsQuery(
    league_id="97108",
    game_code="nhl",
    game_id=427,
    yahoo_consumer_key=CLIENT_ID,
    yahoo_consumer_secret=CLIENT_SECRET
)

In [5]:
LEAGUE_KEY = query.get_league_key(season=None)

In [6]:
league_info = query.get_league_info()

In [7]:
def get_players(player_keys):
    return [
        player 
        for i in range(int(len(player_keys) / 25) + 1)
        for player in query.query(
            f"https://fantasysports.yahooapis.com/fantasy/v2/league/{league_info.league_key}/players;player_keys={','.join(player_keys[i*25:min((i+1)*25, len(player_keys))])};start={i*25}/stats",
            ["league", "players"]
        )
    ]

In [8]:
TEAMS = query.get_league_teams()

In [9]:
LEAGUE_SETTINGS = query.get_league_settings()
SCORING_CATEGORIES = LEAGUE_SETTINGS.stat_categories.stats
for i in range(len(SCORING_CATEGORIES)):
    SCORING_CATEGORIES[i] = SCORING_CATEGORIES[i].clean_data_dict()
SCORING_CATEGORIES = pd.DataFrame(data=SCORING_CATEGORIES)

SCORING_MODIFIERS = data=LEAGUE_SETTINGS.stat_modifiers.stats
for i in range(len(SCORING_MODIFIERS)):
    SCORING_MODIFIERS[i] = SCORING_MODIFIERS[i].clean_data_dict()
SCORING_MODIFIERS = pd.DataFrame(data=SCORING_MODIFIERS)

SCORING_SETTINGS = pd.merge(left=SCORING_CATEGORIES, right=SCORING_MODIFIERS, on='stat_id', how='inner')

Draft Info

In [10]:
draft = query.get_league_draft_results()

Seperate Draft into by Team

In [11]:
draft = pd.DataFrame(data=draft, columns=['Overall', 'Round', 'Team ID', 'Player ID'])

team_drafts = {}
team_keys = []
for ID in range(1, draft['Team ID'].nunique()+1):
    team_keys = [LEAGUE_KEY + '.t.' + str(ID) for ID in range(1, draft['Team ID'].nunique()+1)] 
team_keys = tuple(team_keys)

In [12]:
team_drafts = {team_key : draft[draft['Team ID'] == team_key] for team_key in team_keys}

Total Points for each Team

In [ ]:
def get_stat_df(stat_dict):
    for 

In [96]:
for draft in team_drafts:
    1

In [113]:
total_draft_points = dict.fromkeys(team_keys, 0)
for team in team_drafts:
    # pull each team draft 
    draft = list(team_drafts[team]['Player ID'])
    stats = get_players(player_keys=draft)
    
    for j in range(len(draft)):
        # pull each player from the draft
        # pull each player's stats
        player_stats = stats[j].player_stats.clean_data_dict()['stats']
        
        for i in range(len(player_stats)):
            # pull league scoring settings
            # multiply league settings by player's stats
            stat = player_stats[i]['stat'].clean_data_dict()
            if stat['stat_id'] in list(SCORING_SETTINGS['stat_id']):
                total_draft_points[team] += round(stat['value'] * list(SCORING_SETTINGS['value'])[list(SCORING_SETTINGS['stat_id']).index(stat['stat_id'])], 1)

# for draft in team_drafts:

# mcdavid_stats = query.get_player_stats_for_season(player_key="427.p.6743") #.player_stats.clean_data_dict()['stats']
# mcdavid_stats[0]['stat'].value
# print(mcdavid_stats)
# print(mcdavid_stats.clean_data_dict())



Assign Best and Worst Draft

In [112]:
worst_draft_team_key = min(total_draft_points, key=total_draft_points.get)
best_draft_team_key = max(total_draft_points, key=total_draft_points.get)

print('Worst Draft: ' + worst_draft_team_key)
print('Best Draft: ' + best_draft_team_key)

Worst Draft: 427.l.97108.t.4
Best Draft: 427.l.97108.t.6


Playground

In [50]:
# Mcdavid = query.get_player_stats_for_season("453.p.6743")

In [51]:
# Mcdavid.stats

In [ ]:
query.get_league_matchups_by_week(18)

[Matchup({
   "is_consolation": 0,
   "is_playoffs": 0,
   "status": "preevent",
   "teams": [
     {
       "team": {
         "draft_position": 1,
         "has_draft_grade": 0,
         "is_owned_by_current_login": 1,
         "league_scoring_type": "headpoint",
         "managers": {
           "manager": {
             "email": "amtadam14@hotmail.ca",
             "felo_score": 571,
             "felo_tier": "bronze",
             "guid": "NGSXRXJK2U3UJVTCC2PO3L2A6M",
             "image_url": "https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg",
             "is_commissioner": 0,
             "is_current_login": 1,
             "manager_id": 1,
             "nickname": "adam"
           }
         },
         "name": "McFuck",
         "number_of_moves": 38,
         "number_of_trades": 0,
         "roster_adds": {
           "coverage_type": "week",
           "coverage_value": 17,
           "value": 0
         },
         "team_id": 1,
         "team_key": "453.l.6

In [ ]:
transactions = query.get_league_transactions()

In [ ]:
adds = {}
for transaction in transactions:
    if transaction.type == "drop":
        print(transaction.players)

[Player({
  "display_position": "D",
  "editorial_team_abbr": "CBJ",
  "name": {
    "ascii_first": "Ivan",
    "ascii_last": "Provorov",
    "first": "Ivan",
    "full": "Ivan Provorov",
    "last": "Provorov"
  },
  "player_id": 6749,
  "player_key": "453.p.6749",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C,RW",
  "editorial_team_abbr": "UTA",
  "name": {
    "ascii_first": "Nick",
    "ascii_last": "Schmaltz",
    "first": "Nick",
    "full": "Nick Schmaltz",
    "last": "Schmaltz"
  },
  "player_id": 6386,
  "player_key": "453.p.6386",
  "position_type": "P",
  "transaction_data": {
    "destination_type": "waivers",
    "source_team_key": "453.l.67269.t.4",
    "source_team_name": "(",
    "source_type": "team",
    "type": "drop"
  }
})]
[Player({
  "display_position": "C",
  "editori

In [ ]:
print(transactions[1])

Transaction({
  "players": {
    "player": {
      "display_position": "C,RW",
      "editorial_team_abbr": "UTA",
      "name": {
        "ascii_first": "Nick",
        "ascii_last": "Schmaltz",
        "first": "Nick",
        "full": "Nick Schmaltz",
        "last": "Schmaltz"
      },
      "player_id": 6386,
      "player_key": "453.p.6386",
      "position_type": "P",
      "transaction_data": {
        "destination_type": "waivers",
        "source_team_key": "453.l.67269.t.4",
        "source_team_name": "(",
        "source_type": "team",
        "type": "drop"
      }
    }
  },
  "status": "successful",
  "timestamp": 1739232690,
  "transaction_id": 525,
  "transaction_key": "453.l.67269.tr.525",
  "type": "drop"
})
